# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse_table', engine)
X = df.message
y = df[df.columns[4:]] 
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

C:\Users\marco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\marco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\marco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\marco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\marco\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
predict_y = pipeline.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(y_test[category], predict_y[:, i]))

related
              precision    recall  f1-score   support

           0       0.59      0.37      0.46      1472
           1       0.83      0.92      0.87      5037
           2       0.40      0.13      0.20        45

    accuracy                           0.79      6554
   macro avg       0.61      0.48      0.51      6554
weighted avg       0.77      0.79      0.78      6554

request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5432
           1       0.85      0.38      0.52      1122

    accuracy                           0.88      6554
   macro avg       0.87      0.68      0.73      6554
weighted avg       0.88      0.88      0.86      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\marco\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6422
           1       0.67      0.03      0.06       132

    accuracy                           0.98      6554
   macro avg       0.82      0.51      0.52      6554
weighted avg       0.97      0.98      0.97      6554

tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6516
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6486
           1       0.00      0.00      0.00        68

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

sho

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x00000138B079CAE8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_fea

In [8]:
parameters = {
    "clf__estimator__n_estimators": [50, 100, 200],
    "clf__estimator__min_samples_split": [2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=2)

In [9]:
cv.fit(X_train, y_train)

C:\Users\marco\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 30.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [10]:
cv.cv_results_

{'mean_fit_time': array([244.39142871, 482.61681517, 939.17742689, 221.6105175 ,
        430.80701669, 749.46771224, 186.78299967, 349.99866303,
        504.59733653]),
 'std_fit_time': array([ 4.34666385,  4.13949687, 10.0400549 ,  4.86090495, 12.31733966,
        42.64167563,  3.97487405,  7.92634075, 35.46267799]),
 'mean_score_time': array([15.84537506, 27.45133265, 46.32666699, 17.58414276, 24.70099719,
        38.31666533, 14.60933415, 21.95700057, 27.52466059]),
 'std_score_time': array([0.37667882, 2.93501124, 1.41597677, 0.43435845, 1.5683955 ,
        5.39757948, 1.02775211, 0.85459401, 1.02308442]),
 'param_clf__estimator__min_samples_split': masked_array(data=[2, 2, 2, 3, 3, 3, 4, 4, 4],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_clf__estimator__n_estimators': masked_array(data=[50, 100, 200, 50, 100, 200, 50, 100, 200],
              mask=[False, False, 

In [11]:
#best parameters result
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}


In [12]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_pred = optimised_model.predict(X_test)

In [14]:
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(y_test[category], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.69      0.28      0.40      1472
           1       0.82      0.96      0.88      5037
           2       0.50      0.13      0.21        45

    accuracy                           0.80      6554
   macro avg       0.67      0.46      0.50      6554
weighted avg       0.79      0.80      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5432
           1       0.89      0.45      0.60      1122

    accuracy                           0.90      6554
   macro avg       0.90      0.72      0.77      6554
weighted avg       0.90      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\marco\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6115
           1       0.92      0.19      0.32       439

    accuracy                           0.94      6554
   macro avg       0.93      0.60      0.64      6554
weighted avg       0.94      0.94      0.93      6554

food
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5789
           1       0.90      0.41      0.56       765

    accuracy                           0.93      6554
   macro avg       0.91      0.70      0.76      6554
weighted avg       0.92      0.93      0.91      6554

shelter
              precision    recall  f1-score   support


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [15]:
pkl_filename = 'classifier.pkl'
with open(pkl_filename, 'wb') as file:  
    pickle.dump(optimised_model, file)

- Classifier is too big for github (1.7 GB). I'll use other params in order to reduce file size.

In [16]:
#best parameters result
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 200}


- Let's test new size

In [17]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters = {
        'clf__estimator__n_estimators': [20],
        'clf__estimator__min_samples_split': [2],
    
    }

final_model = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=3)

In [18]:
final_model.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [19]:
y_pred = final_model.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(y_test[category], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.62      0.32      0.42      1472
           1       0.82      0.94      0.88      5037
           2       0.58      0.16      0.25        45

    accuracy                           0.80      6554
   macro avg       0.67      0.47      0.52      6554
weighted avg       0.77      0.80      0.77      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5432
           1       0.86      0.41      0.56      1122

    accuracy                           0.89      6554
   macro avg       0.87      0.70      0.75      6554
weighted avg       0.88      0.89      0.87      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\marco\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6342
           1       0.64      0.07      0.12       212

    accuracy                           0.97      6554
   macro avg       0.80      0.53      0.55      6554
weighted avg       0.96      0.97      0.96      6554

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

water
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6115
           1       0.90      0.23      0.37       439

    accuracy                           0.95      6554
   macro avg       0.93      0.62      0.67      6554
weighted avg       0.94      0.95      0.93      6554

food
              precision    recall  f1-score   supp

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6187
           1       0.72      0.06      0.11       367

    accuracy                           0.95      6554
   macro avg       0.84      0.53      0.54      6554
weighted avg       0.93      0.95      0.92      6554

direct_report
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      5282
           1       0.84      0.37      0.51      1272

    accuracy                           0.86      6554
   macro avg       0.85      0.67      0.72      6554
weighted avg       0.86      0.86      0.84      6554



In [20]:
pkl_filename = 'classifier_2.pkl'
with open(pkl_filename, 'wb') as file:  
    pickle.dump(final_model, file)

- This reduces size to 170MB

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.